# Work with textual data: encyclicals example

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import os
from collections import defaultdict

In [3]:
folder = '/Users/flint/Data/vatican/data'
data_collection = {}
for subdir in os.listdir(folder):
    pope = subdir
    data_collection[pope] = {}
    path = os.path.join(folder, pope, 'encyclicals')
    for doc in os.listdir(path):
        subpath = os.path.join(path, doc, 'it', '{}.txt'.format(doc))
        with open(subpath, 'r') as infile:
            content = infile.read()
        data_collection[pope][doc] = content

In [4]:
print(list(data_collection.keys()))

['Pius X', 'John XXIII', 'John Paul II', 'Leo XIII', 'Benedict XV', 'Francis I', 'Pius XII', 'Paul VI', 'Benedict XVI', 'Pius XI']


In [5]:
print(list(data_collection['Paul VI'].keys()))

['Populorum', 'Humanae Vitae', 'Sacerdotalis', 'Christi Matri', 'Mense Maio', 'Mysterium', 'Ecclesiam']


In [6]:
print(data_collection['Paul VI']['Humanae Vitae'][:1000])

[ LETTERA ENCICLICA DEL SOMMO PONTEFICE PAOLO PP. VI HUMANAE VITAE _ 1. Il gravissimo dovere di trasmettere la vita umana, per il quale gli sposi sono liberi e responsabili collaboratori di Dio creatore, è sempre stato per essi fonte di grandi gioie, le quali, tuttavia, sono talvolta accompagnate da non poche difficoltà e angustie. In tutti i tempi l'adempimento di questo dovere ha posto alla coscienza dei coniugi seri problemi, ma col recente evolversi della società, si sono prodotti mutamenti tali da far sorgere nuove questioni, che la chiesa non può ignorare, trattandosi di materia che tanto da vicino tocca la vita e la felicità degli uomini. I. ASPETTI NUOVI DEL PROBLEMA E COMPETENZA DEL MAGISTERO 2. I cambiamenti avvenuti sono infatti di grande importanza e di vario genere. Si tratta anzitutto del rapido sviluppo demografico, per il quale molti manifestano il timore che la popolazione mondiale cresca più rapidamente delle risorse a disposizione, con crescente angustia di tante fam

## Tokenization and frequency

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [8]:
import spacy
nlp = spacy.load('it_core_news_lg')

In [9]:
document = data_collection['Paul VI']['Humanae Vitae']

### Space tokenizer

In [10]:
"questa è un'albicocca".split(), "questa è albicocca".split()

(['questa', 'è', "un'albicocca"], ['questa', 'è', 'albicocca'])

### Regex tokenizer

In [11]:
word_tokenize("testo in cui si parla di albicocca.", language='italian')

['testo', 'in', 'cui', 'si', 'parla', 'di', 'albicocca', '.']

In [12]:
word_tokenize("testo in cui si parla di albicocche.", language='italian')

['testo', 'in', 'cui', 'si', 'parla', 'di', 'albicocche', '.']

### Syntactic tokenizer

In [13]:
from spacy.displacy import render

In [14]:
doc = nlp(document)

In [15]:
sentences = [sentence for sentence in doc.sents]

In [16]:
print(sentences[21])

Si chiede anche se, dato l'accresciuto senso di responsabilità dell'uomo moderno, non sia venuto per lui il momento di affidare alla sua ragione e alla sua volontà, più che ai ritmi biologici del suo organismo, il compito di trasmettere la vita.


In [17]:
for token in sentences[21]:
    print(token, token.lemma_, token.pos_)

Si si PRON
chiede chiedere VERB
anche anche ADV
se se SCONJ
, , PUNCT
dato dare VERB
l' il DET
accresciuto accresciuto ADJ
senso senso NOUN
di di ADP
responsabilità responsabilità NOUN
dell' di il ADP
uomo uomo NOUN
moderno moderno ADJ
, , PUNCT
non non ADV
sia essere AUX
venuto venire VERB
per per ADP
lui lui PRON
il il DET
momento momento NOUN
di di ADP
affidare affidare VERB
alla a il ADP
sua suo DET
ragione ragione NOUN
e e CCONJ
alla a il ADP
sua suo DET
volontà volontà NOUN
, , PUNCT
più più ADV
che che SCONJ
ai a il ADP
ritmi ritmo NOUN
biologici biologico ADJ
del di il ADP
suo suo DET
organismo organismo NOUN
, , PUNCT
il il DET
compito compito NOUN
di di ADP
trasmettere trasmettere VERB
la il DET
vita vita NOUN
. . PUNCT


In [18]:
print(sentences[21])
render(sentences[21])

Si chiede anche se, dato l'accresciuto senso di responsabilità dell'uomo moderno, non sia venuto per lui il momento di affidare alla sua ragione e alla sua volontà, più che ai ritmi biologici del suo organismo, il compito di trasmettere la vita.


## Processo di tokenizzazione
- Filtro: togliamo punteggiatura PUNCT, articoli DET, ADP
- Con lemmatizzazione

In [19]:
def tokenize(text, lemmatize=True, filter_pos=None):
    bow = defaultdict(lambda: 0)
    doc = nlp(text)
    for sentence in doc.sents:
        for token in sentence:
            if lemmatize:
                word = token.lemma_
            else:
                word = token.text
            if filter_pos is None:
                bow[word] += 1
            else:
                if token.pos_ not in filter_pos:
                    bow[word] += 1
    return pd.Series(bow)

In [20]:
bow = tokenize(data_collection['Paul VI']['Humanae Vitae'], 
               filter_pos=['PUNCT', 'DET', 'ADP'])

In [21]:
bow.sort_values(ascending=False).head(20)

e            220
essere       127
che          114
non           70
si            50
vita          42
uomo          40
potere        36
chiesa        36
ma            33
umano         31
coniugale     31
amore         29
avere         28
legge         27
anche         27
più           27
sposo         25
dovere        24
se            24
dtype: int64

In [22]:
corpus_index = {}
popes = ['John XXIII', 'Paul VI']
for pope, corpus in data_collection.items():
    if pope in popes:
        corpus_index[pope] = {}
        for document, content in corpus.items():
            freq = tokenize(content)
            corpus_index[pope][document] = freq

In [23]:
hum_vit = corpus_index['Paul VI']['Humanae Vitae']
pax = corpus_index['John XXIII']['Pacem']

In [24]:
hum_vit.sort_values(ascending=False)['uomo'] / hum_vit.sum()

0.006091061367443277

In [25]:
hum_vit.sort_values(ascending=False)['che'] / hum_vit.sum()

0.017664077965585504

In [26]:
pax.sort_values(ascending=False)['uomo'] / pax.sum()

0.002189090048514969

In [27]:
pax.sort_values(ascending=False)['che'] / pax.sum()

0.01792687255946042

In [28]:
hum_vit_norm = hum_vit / hum_vit.sum()

In [29]:
hum_vit_norm.sort_values(ascending=False)

,                0.067611
il               0.061824
di il            0.044160
e                0.033501
di               0.030303
                   ...   
assolutamente    0.000152
interruzione     0.000152
aborto           0.000152
procurare        0.000152
]                0.000152
Length: 1381, dtype: float64

## Document frequency

In [30]:
docu_freq = defaultdict(lambda: 0)
for pope, corpus in corpus_index.items():
    for document, idx in corpus.items():
        for word in idx.keys():
            docu_freq[word] += 1
DF = pd.Series(docu_freq)

In [31]:
DF.sort_values(ascending=False).head(20)

[            15
a            15
su il        15
perciò       15
non          15
altro        15
preghiera    15
motivo       15
in           15
grave        15
dovere       15
essere       15
4            15
chiesa       15
tutto        15
vostro       15
dio          15
si           15
cristiano    15
2            15
dtype: int64

In [32]:
(15 / DF).sort_values(ascending=False)

1Cor             15.0
assistono        15.0
adire            15.0
soddisfare li    15.0
ìmpari           15.0
                 ... 
uno               1.0
che               1.0
lo                1.0
anche             1.0
[                 1.0
Length: 10672, dtype: float64